# Исследование надёжности заёмщиков


### План выполнения проекта ###

 [Шаг 1. Изучение общей информации](#step1)
 
 
 [Шаг 2. Предобработка данных](#step2)
 
 
 [Шаг 3. Влияние различных факторов на факт погашения кредита в срок](#step3)
 
 
 [Шаг 4. Выводы](#step4)

## <a id="step1">Шаг 1. Изучение общей информации </a>

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4  

In [3]:
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В данной таблице 21525 строчек и 12 колонок, в колонках `days_employed` и `total_income` имеются пропущенные значения, причем в одинаковом количестве. Предполагаю, что пропуски в одинаковых строках, так как если клиент не предоставил информацию о трудовом стаже, то и подтвержденных данных о доходе тоже вряд ли будет. Скорее, это те, кто работает неофициально.

В данной таблице в колонке `days_employed` мы также видим, что значения отрицательные в некоторых колонках. Возможно, это техническая ошибка при считывании информации.

## <a id="step2">Шаг 2. Предобработка данных </a>

### Обработка пропусков

In [4]:
# Исследуем пропуски на случайность, проходимся по всем столбцам и прсмотрим, может, есть закономерность?
print(data[data['total_income'].isnull()]['children'].value_counts(normalize = True).head()) 
print(data['children'].value_counts(normalize = True).head()) 

print(data[data['total_income'].isnull()]['income_type'].value_counts(normalize = True).head())
print(data['income_type'].value_counts(normalize = True).head())

print(data[data['total_income'].isnull()]['education'].value_counts(normalize = True).head())
print(data['education'].value_counts(normalize = True).head())

print(data[data['total_income'].isnull()]['family_status'].value_counts(normalize = True).head())
print(data['family_status'].value_counts(normalize = True).head())

print(data[data['total_income'].isnull()]['purpose'].value_counts(normalize = True).head())
print(data['purpose'].value_counts(normalize = True).head())

print(data[data['total_income'].isnull()]['debt'].value_counts(normalize = True).head())
print(data['debt'].value_counts(normalize = True).head())

0     0.661914
1     0.218491
2     0.093836
3     0.016559
20    0.004140
Name: children, dtype: float64
0     0.657329
1     0.223833
2     0.095470
3     0.015331
20    0.003531
Name: children, dtype: float64
сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64
сотрудник      0.516562
компаньон      0.236237
пенсионер      0.179141
госслужащий    0.067782
безработный    0.000093
Name: income_type, dtype: float64
среднее                0.647654
высшее                 0.228151
СРЕДНЕЕ                0.030819
Среднее                0.029899
неоконченное высшее    0.025299
Name: education, dtype: float64
среднее                0.638792
высшее                 0.219187
СРЕДНЕЕ                0.035865
Среднее                0.033031
неоконченное высшее    0.031034
Name: education, dtype: float64
женат / замужем          0.568997
гражданский брак         0.203312
Не женат

Пропуски не отражают какую-либо закономерность, можно считать эти пропуски случайными и заполнять характерными значениями

In [5]:
print(data[data['total_income'].isnull()]['days_employed'].value_counts(normalize = True, dropna = False))
print(data['days_employed'].value_counts(normalize = True, dropna = False).head())

print(data['days_employed'])
print(data['days_employed']/365)

NaN    1.0
Name: days_employed, dtype: float64
 NaN            0.100999
-1645.463049    0.000046
-6620.396473    0.000046
-1238.560080    0.000046
-3047.519891    0.000046
Name: days_employed, dtype: float64
0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64
0        -23.116912
1        -11.026860
2        -15.406637
3        -11.300677
4        932.235814
            ...    
21520    -12.409087
21521    942.294258
21522     -5.789991
21523     -8.527347
21524     -5.437007
Name: days_employed, Length: 21525, dtype: float64


В строчках, где есть пропущенный доход, дни стажа также пропущены.

Отрицательные значения в `days_employed` появились в результате технической ошибки при считывании данных (тире засчитался как минус). А значения у пенсионеров скорее, записывалось в другом формате - количество лет и рядом количество дней, так как при делении на дней в году мы получили неадекватно большие числа.

Приведем стобец `days_employed` к абсолютным значениям и приведем нереальные данные к реальным:

In [6]:
data['days_employed'] = data['days_employed'].abs()

def days_right(days):
    if days > 36500:
        result = (days//10000)*365 + days % 10000 
        return result
    else:
        return days
data['days_employed'] = data['days_employed'].apply(days_right)



Теперь нужно заполнить пропуски. Для этого нужно взять медиану по признакоу income_type

In [7]:
grouped_data_days = data.groupby('income_type')['days_employed'].median()
print(grouped_data_days)

income_type
безработный        19553.652744
в декрете           3296.759962
госслужащий         2689.368353
компаньон           1547.382223
пенсионер          18125.532527
предприниматель      520.848083
сотрудник           1574.202821
студент              578.751554
Name: days_employed, dtype: float64


Выявился неожиданно большой опыт работы у безработных(более 100 лет), уточним данные:

In [8]:
display(data[data['income_type']=='безработный'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,19569.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,19537.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


Перезапишем данные о днях занятости у безработных. Для этого найдем медиану отработанных дней у людей того же возраста:

In [9]:
median_days31 = data[data['dob_years']==31]['days_employed'].median()
median_days45 = data[data['dob_years']==45]['days_employed'].median()
data.loc[14798, 'days_employed'] = median_days45
data.loc[3133, 'days_employed'] = median_days31

Переходим к заполнению пропущенных значений в days_employed и total_income:

In [10]:
row_unique_income_type = data['income_type'].unique()
grouped_data_income = data.groupby('income_type')['total_income'].median()

def fill_nan(data):
    for x in row_unique_income_type:
        data.loc[data['income_type']==x, 'days_employed'] = data.loc[data['income_type']==x, 'days_employed'].fillna(grouped_data_days.loc[x])
        data.loc[data['income_type']==x, 'total_income'] = data.loc[data['income_type']==x, 'total_income'].fillna(grouped_data_income.loc[x])
fill_nan(data)

data.info()
# Пропусков нет.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [11]:
# Проверяем другие колонки на наличие возможных ошибок. Для этого посмотрим уникальные значения и их число
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


-1 ребенок - это вряд ли. Так же как и 20. Это возможно в теории, но не в 76 случаях из 21525. Скорее всего, это 2 детей.

In [12]:
data['children'] = data['children'].abs()
data['children'].replace(20, 2, inplace=True)
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Проверяем колонку с возрастом клиентов, видим значение 0, что невозможно. заменяем 0 на средний возраст клиентов

In [13]:
print(data['dob_years'].value_counts())
avg_dob_years = int(data['dob_years'].mean())
data['dob_years'].replace(0, avg_dob_years, inplace=True)
print(avg_dob_years)
print(data['dob_years'].value_counts())



35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
43
35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    2

В колонках `days_employed` и `total_income` имеются пропущенные значения, причем в одинаковом количестве и в одинаковых строчках. Так как нет определенной закономерности в пропусках, необходимо пропущенные значения в двух колонках заменить медианой по признаку `income_type` (именно по этому признаку таблица уже категоризирована). Заменяем медианой, так как средний ежемесячный доход и трудовой стаж - это количественные переменные и данные показатели могут очень сильно разниться.

Очень много отрицательных значений в колонке `days_employed`, положительные значения - это количество дней стажа в шестизначном выражении. То есть, в графе Количество дней стажа было 6 ячеек для заполнения, при указании 5значного числа, программа автоматически ставит "-" впереди. Данная сложность была исправлена методом `abs()`, который возвращает числам их абсолютное значение.

Такая же проблема в колонке `children,` более того - мы видим значение - 20 детей. Это возможно в теории, но у ограниченного количества людей, может быть у двух-трех. Стоит проверить все строки. Заменили значение на 2, что вероятнее.
Также в колонке `dob_years` обнаружен клиент с возрастом 0. Заменим на средний возраст.

Все пропуски заменены на расчетные значения (медиана или среднее).

### Замена типа данных

In [14]:
# Заменяем данные вещественного типа в данных колонках на целочисленный
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Необходимо было заменить вещественный тип данных в строках на целочисленный. Был применен метод astype(), так как он переводит заданный тип данных к определенному типу данных, в то время как метод to_numeric() переводит строковые данные к float и int.

### Обработка дубликатов

Проверяем на дубликаты во всех колонках c объектами. Применяем метод value_counts() поскольку в нашей таблице нет уникальных значений, только общие характеристики клиентов.

In [15]:
print(data['gender'].value_counts())  
print(data['income_type'].value_counts())  
print(data['purpose'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недв

Итак, пол XNA возможен в современном мире, оставим все как есть.

В типах дохода тоже все в порядке, а вот в колонке `purpose` необходимо произвести лемматизацию, много дубликатов вследствии отсутствия категорий целей.

In [16]:
print(data['education'].value_counts())

# Применяем метод lower() к столбцу education для удаления дубликатов
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [17]:
print(data['family_status'].value_counts())

# Применяем тот же метод и к столбцу family_status для удобства представления данныз
data['family_status'] = data['family_status'].str.lower()
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Рассмотрим полный дубликаты данных.

In [18]:
print(data.duplicated().sum())
display(data[data.duplicated()])

71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,18125,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,1574,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,18125,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,18125,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,18125,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
21032,0,18125,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием
21132,0,1574,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью
21281,1,1574,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости


In [19]:
# проанализируем возможные причины возникновения полных дубликатов
print(data[data.duplicated()]['income_type'].value_counts(normalize=True))
print(data[data.duplicated()]['days_employed'].value_counts(normalize=True))
print(data[data.duplicated()]['total_income'].value_counts(normalize=True))
print(data[data.duplicated()]['debt'].value_counts(normalize=True))
print(data[data.duplicated()]['children'].value_counts(normalize=True))
print(data[data.duplicated()]['education'].value_counts(normalize=True))
print(data[data.duplicated()]['family_status'].value_counts(normalize=True))
print(data[data.duplicated()]['purpose'].value_counts(normalize=True))
print(data[data.duplicated()]['dob_years'].value_counts(normalize=True))

сотрудник      0.492958
пенсионер      0.380282
компаньон      0.098592
госслужащий    0.028169
Name: income_type, dtype: float64
1574     0.492958
18125    0.380282
1547     0.098592
2689     0.028169
Name: days_employed, dtype: float64
142594    0.492958
118514    0.380282
172357    0.098592
150447    0.028169
Name: total_income, dtype: float64
0    1.0
Name: debt, dtype: float64
0    0.816901
1    0.140845
2    0.042254
Name: children, dtype: float64
среднее    0.859155
высшее     0.140845
Name: education, dtype: float64
женат / замужем          0.577465
гражданский брак         0.366197
не женат / не замужем    0.042254
вдовец / вдова           0.014085
Name: family_status, dtype: float64
сыграть свадьбу                          0.126761
на проведение свадьбы                    0.126761
свадьба                                  0.084507
ремонт жилью                             0.070423
заняться образованием                    0.056338
сделка с подержанным автомобилем         0.04225

Есть четкое распределение по типу дохода, дням стажа и ежемесячному доходу,что, очевидно, связано с заполнением пропусков. 
Распределение по остальным признаком не такое четкое, у нас нет сведений, что каждая запись соответствует одному пользователю.
А образование, цель кредита, количество лет и детей - это не уникальные значения, они могут совпадать у нескольких человек.



Благодаря методу `value_counts()` видим, что дубликаты есть в колонках `purpose` и `education`. Скорее всего, в других колонках при заполнении информации о клиентах, были расписаны варианты ответов, среди которых можно было отметить подходящий. Дубликаты в колонке `education` возникли из-за разных регистрах в ответах, в колонке purpose - из-за отсутствия категоризации вариантов ответов.

В колонке `family_status` - заглавная буква Н в "не замужем", скорее, из-за человеческого фактора.

В колонке `gender` имеется одна строчка с полом XNA. Это либо ошибка, либо человек трансгендер/без определенного пола.

Приведением к нижнему регистру методом `lower()` мы исправили положение в колонках `education и family_status`. Далее необходимо категоризировать фразы в `purpose` для удаления дубликатов.

Мы также рассмотрели и проанализировали все полные дубликаты в таблице. Есть четкое распределение по типу дохода, дням стажа и ежемесячному доходу,что, очевидно, связано с заполнением пропусков. 
Распределение по остальным признаком не такое четкое, у нас нет сведений, что каждая запись соответствует одному пользователю.
А образование, цель кредита, количество лет и детей - это не уникальные значения, они могут совпадать у нескольких человек.
Таким образом, можно оставить эти дубликаты. 71 из 21525 это около 0,3 процента. Даже если половина из этого количества все же дубликаты (0,15%), то это мало повлияет на результат.

### Лемматизация

In [20]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Лемматизацию столбца удобнее проводить с помощью библиотеки pymastem3 для нахождения слов к его словарной форме, так как у нас присутствуют фразы. Создадим отдельно столбец с леммами, а также столбец с категориями целей кредита.

In [21]:
m = Mystem()

# Создаем новую колонку с леммами колонки purpose
def purpose_lemmas(purpose):
    return m.lemmatize(purpose)
data['purpose_lemmas'] = data['purpose'].apply(purpose_lemmas)
print(data['purpose_lemmas'])

# С помощью метода Counter() удобно подсчитать количество часто встречающихся слов в леммах для определения 
# основных типов целей клиентов
from collections import Counter
lemmas = []
for x in data['purpose_lemmas']:
    lemmas += x
print(Counter(lemmas))

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: purpose_lemmas, Length: 21525, dtype: object
Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'занима

In [22]:
# Создаем функцию для создания нового столбца type_purpose на основе столбца с леммами
def words(row):
    purpose = row['purpose_lemmas']
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'недвижимость' in purpose:
        if 'коммерческий' in purpose:
            return 'коммерческая недвижимость'
    return 'жилая недвижимость'
data['type_purpose'] = data.apply(words, axis = 1)

# Проверяем, все ли строки заполнены, нет ли пропусков
print(data['type_purpose'].value_counts())
print(data['type_purpose'].value_counts().sum())
data.info()


жилая недвижимость           9525
автомобиль                   4315
образование                  4022
свадьба                      2348
коммерческая недвижимость    1315
Name: type_purpose, dtype: int64
21525
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
purpose_lemmas      21525 non-null object
type_purpose        21525 non-null object
dtypes: int64(7), object(7)
memory usage: 2.3+ MB


Так как мы имеем дело не с одиночными словами, а с фразами, в которых много лишних слов, необходимо использовать метод из библиотеки pymystem3 для удобства. Мы определили все ключевые фразы, далее - все типы кредита по целям. Это `Автомобиль`, `Коммерческая недвижимость` и `Жилая недвижимость`, `Свадьба`, `Образование`. 

### Категоризация данных

Для категоризации по наличию детей напишем функцию и применим ее к столбцу с количеством детей для создания столбца с значениями "есть дети" и "нет детей"

In [23]:
print(data['children'].value_counts())

def children_or_not(row):
    children = row['children']
    if children !=0:
        return 'есть дети'
    return 'нет детей'

data['children_or_not'] = data.apply(children_or_not, axis=1)

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Для категоризации по уровню дохода  напишем функцию и применим ее к столбцу с ежемесячным дохода для создания столбца с значениями : низкий, средний, высокий и очень высокий. 

In [24]:

print(data['total_income'].describe().apply(lambda x: format(x, 'f')))
print(data['total_income'].quantile(q=[0.25,0.5,0.75]))

def total_income_type(row):
    total_income = row['total_income']
    if 55000 >= total_income < 107798:
        return 'низкий'
    if 107798 <= total_income < 142594:
        return 'средний'
    if 142594 <= total_income < 195549:
        return 'высокий'
    return 'очень высокий'
data['total_income_type'] = data.apply(total_income_type, axis=1)
display(data.head())
        




count      21525.000000
mean      165224.816028
std        98043.667724
min        20667.000000
25%       107798.000000
50%       142594.000000
75%       195549.000000
max      2265604.000000
Name: total_income, dtype: object
0.25    107798.0
0.50    142594.0
0.75    195549.0
Name: total_income, dtype: float64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,type_purpose,children_or_not,total_income_type
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилая недвижимость,есть дети,очень высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,есть дети,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилая недвижимость,нет детей,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,есть дети,очень высокий
4,0,12676,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,нет детей,высокий


Категоризировали таблицу по тем признакам, по которым нам предстоит проанализировать данные. 

Данные в таблице по целям кредита и семейному статусу уже категоризиованы. 

Данные по наличию детей не были категоризированы, поэтому мы взяли все значения больше 0 и возвратили значение "есть дети" в новый столбец. И наоборот.

Так как данные по доходу предствлены с огромным размахом, а задача стоит проанализировать данные именно по уровню дохода, а не по типу дохода, необходимо было категоризировать данные по признаку - уровень дохода. Для это мы сгруппировали таблицу по типу дохода и посмотрели средний доход по этому признаку. Далее - выявили низкий, средний, высокий и очень высокие доходы по частоте встречаний и логическому подходу(декретные и стипендия являются низкими доходами по сравнению с остальными и т.п.)

## <a id="step3">Шаг 3. Влияние различных факторов на факт погашения кредита в срок</a>

### - Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
# Создадим сгруппирированную таблицу для того, чтобы удобнее было найти процент кредитов, незакрытых в срок
grouped_data_children = data.groupby('children_or_not').agg({'debt': ['count', 'sum']})
grouped_data_children['conversion_debt'] = grouped_data_children['debt']['sum'] / grouped_data_children['debt']['count'] * 100
display(grouped_data_children)
pivot_children = data.pivot_table(index='children_or_not', values='debt', aggfunc=['count','sum'])
display(pivot_children)
pivot_children['conversion']=pivot_children['sum']['debt']/pivot_children['count']['debt'] * 100
display(pivot_children)

debt       conversion_debt
                 count   sum                
children_or_not                             
есть дети         7376   678        9.191974
нет детей        14149  1063        7.512898

,count,sum
,debt,debt
children_or_not,,
есть дети,7376,678
нет детей,14149,1063


,count,sum,conversion
,debt,debt,
children_or_not,,,
есть дети,7376,678,9.191974
нет детей,14149,1063,7.512898


При наличии детей вероятность возврата кредита в срок снижается на 1,65%. Вероятно, это связано с увеличением расходов на детей и невозможности в некоторых случаях всегда иметь ежемесячный платеж к выплате.

### - Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
print(data['family_status'].value_counts())

# Создадим словарь для хранения меньшого размера данных и сгруппированную таблицу по признакам - семейный статус и айди
data_family_status_dict = data[['family_status', 'family_status_id', 'debt']]
grouped_data_fs = data_family_status_dict.groupby(['family_status_id', 'family_status']).agg({'debt': ['count', 'sum']})
display(grouped_data_fs)
grouped_data_fs['conversion_debt'] = grouped_data_fs['debt']['sum'] / grouped_data_fs['debt']['count'] * 100
display(grouped_data_fs.sort_values('conversion_debt'))

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


debt     
                                        count  sum
family_status_id family_status                    
0                женат / замужем        12380  931
1                гражданский брак        4177  388
2                вдовец / вдова           960   63
3                в разводе               1195   85
4                не женат / не замужем   2813  274

debt      conversion_debt
                                        count  sum                
family_status_id family_status                                    
2                вдовец / вдова           960   63        6.562500
3                в разводе               1195   85        7.112971
0                женат / замужем        12380  931        7.520194
1                гражданский брак        4177  388        9.288963
4                не женат / не замужем   2813  274        9.740491

Самый низкая вероятность появления задолженностей у тех, кто пережил смерть супруга\супруги. Такие случаи возникают преимущественно у тех, кто уже на пенсии. И, конечно, они сами по себе берут мало кредиты. А если и берут, то стараются возвращать. Скорее, это консервативный подход. 

У тех, кто сейчас связан узами брака или уже нет = примерно одинакова вероятность не заплатить кредит в срок. 

А у тех, кто еще не был в ЗАГСе - риск появления задолженностей выше всех.

Думаю, те, кто еще документально не зарегистрировал отношения, попросту привыкли жить преимущественно для себя, нет семейного бюджета. И им сложнее думать о ежемесячных обязательствах. 

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
grouped_data_income = data.groupby('total_income_type').agg({'debt': ['count', 'sum']})

grouped_data_income['conversion_debt'] = grouped_data_income['debt']['sum'] / grouped_data_income['debt']['count'] * 100
display(grouped_data_income.sort_values('conversion_debt'))

debt      conversion_debt
                   count  sum                
total_income_type                            
низкий               580   35        6.034483
очень высокий      10183  778        7.640185
высокий             6343  543        8.560618
средний             4419  385        8.712378

Самый высокий риск возникновения задолженности по кредиту у людей со средним доходом. Такие люди живут средне, привыкли распределять бюджет, но при этом покупая крупные вещи в кредит. Поэтому они привыкают брать кредиты, жить не по средствам и зачастую, не могут заплатить в срок, так как кредитов становится много, а если еще дети и живет один...)

Людям с низким доходом нечасто одобряют кредиты, казалось бы, странно, что они как раз таки более добросовестные. Может, дело в том, что они и не могут жить не по средствам, поэтому берут кредиты на самое нужное сейчас.

А те, кто очень хорошо зарабатывают, находятся посередине. Они берут кредиты тоже, может, они просто забывают про сроки? 

### - Как разные цели кредита влияют на его возврат в срок?

In [28]:
print(data['type_purpose'].value_counts())
data.info()
grouped_data_purpose = data.groupby('type_purpose').agg({'debt': ['count', 'sum']})
grouped_data_purpose['conversion_debt'] = grouped_data_purpose['debt']['sum'] / grouped_data_purpose['debt']['count'] * 100
display(grouped_data_purpose.sort_values('conversion_debt'))

жилая недвижимость           9525
автомобиль                   4315
образование                  4022
свадьба                      2348
коммерческая недвижимость    1315
Name: type_purpose, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 16 columns):
children             21525 non-null int64
days_employed        21525 non-null int64
dob_years            21525 non-null int64
education            21525 non-null object
education_id         21525 non-null int64
family_status        21525 non-null object
family_status_id     21525 non-null int64
gender               21525 non-null object
income_type          21525 non-null object
debt                 21525 non-null int64
total_income         21525 non-null int64
purpose              21525 non-null object
purpose_lemmas       21525 non-null object
type_purpose         21525 non-null object
children_or_not      21525 non-null object
total_income_type    21525 non-null object
dtypes:

debt      conversion_debt
                          count  sum                
type_purpose                                        
жилая недвижимость         9525  683        7.170604
коммерческая недвижимость  1315   99        7.528517
свадьба                    2348  186        7.921635
образование                4022  370        9.199403
автомобиль                 4315  403        9.339513

Люди почти наравне расплачиваются по кредитам на недвижимость двух видов. 
Жилая недвижимость - это необходимость, люди заблаговременно рассчитывают возможность взять кредит и расплатиться, так как никому не хочется остаться без жилья.
Коммерческая недвижимость может сразу приносить доход за счет сдачи, либо использоваться собственником для своих нужд бизнеса, следовательно, обеспечивать бесперебойную выплату кредита за счет дохода фирмы.

Чуть больший процент задолженностей у людей на свадьбу, но некритично. Как правило, сумму дарят родственники или близкие уже непосредственно на празднике.

Самый низкий процент вовремя выплаченных кредитов у тех, кто берет деньги на Образование или Автомобиль. Понятно, что наличие образование не гарантирует ни краткосрочного увеличения доходов, ни долгосрочного. Особенно, если образование очное.
Автомобиль чаще всего покупают люди, потому что это удобно. Но мало кто использует его для получение бОльшего дохода, в основном, машина просто становится еще одним голодным ртом у семьи/человека. Как еще один ребенок)

In [29]:
pivot_data = data.pivot_table(index=['children_or_not', 'type_purpose'], values='debt', aggfunc=['count','sum'])
pivot_data['conversion']=pivot_data['sum']['debt']/pivot_data['count']['debt'] * 100
display(pivot_data.head())


count  sum conversion
                                           debt debt           
children_or_not type_purpose                                   
есть дети       автомобиль                 1463  160  10.936432
                жилая недвижимость         3279  271   8.264715
                коммерческая недвижимость   464   35   7.543103
                образование                1373  141  10.269483
                свадьба                     797   71   8.908407

Люди с детьми, да еще с автомобилем, отдают кредиты самым безобразнейшим образом. Машина сама добавляет много расходов, так еще и поводов поехать куда-либо с детьми появляется куча - начинается пора путешествий и долгих выходных. Это не помогает выплатить кредит в долг.

## <a id="step4">Шаг 4. Выводы </a>


In [30]:
pivot_data_widowers = data.pivot_table(index=['family_status', 'children_or_not', 'total_income_type', 'type_purpose'], values='debt', aggfunc=['count','sum'])
pivot_data_widowers['conversion']=pivot_data_widowers['sum']['debt']/pivot_data_widowers['count']['debt'] * 100
display(pivot_data_widowers.loc[['гражданский брак', 'не женат / не замужем'], 'есть дети', 'средний', 'автомобиль'])
display(pivot_data_widowers.loc['вдовец / вдова', 'нет детей', 'низкий', ['жилая недвижимость', 'коммерческая недвижимость']])

,,,,count,sum,conversion
,,,,debt,debt,
family_status,children_or_not,total_income_type,type_purpose,,,
гражданский брак,есть дети,средний,автомобиль,30,1,3.333333
не женат / не замужем,есть дети,средний,автомобиль,29,6,20.689655


count  \
                                                                            debt   
family_status  children_or_not total_income_type type_purpose                      
вдовец / вдова нет детей       низкий            жилая недвижимость           16   
                                                 коммерческая недвижимость     1   

                                                                            sum  \
                                                                           debt   
family_status  children_or_not total_income_type type_purpose                     
вдовец / вдова нет детей       низкий            жилая недвижимость           0   
                                                 коммерческая недвижимость    0   

                                                                           conversion  
                                                                                       
family_status  children_or_not total_income_type type_purpose                          
вдовец / вдова нет детей       низкий            жилая недвижимость               0.0  
                                                 коммерческая недвижимость        0.0

Самый ненадежный клиент банка - человек без брачных уз с детьми, средним доходом, который хочет купить автомобиль. Но если он завтра пойдет с партнером в ЗАГС и будет использовать автомобиль как средство заработка, то вполне себе ничего.

А самый надежный - вдовец без детей с низким доходом, который хочет купить себе недвижимость. Что выглядит очень странно, с учетом того, что он вряд ли рассчитается с кредитом в срок, в силу того, что цена на недвижимость высока, доход низкий, а срок выплаты будет очень долгим, и всей жизни не хватит.

Проанализировав данные в сводной таблице, мы можем увидеть, что:

1) люди, находящиеся в гражданском браке, более ответственны, не смотря на наличие детей, среднего дохода и кредитом на автомобиль, нежели люди неженатые и незамужние. А вот последние показывают очень плохой результат - только 79,3% возвращают кредит в срок по сравнению с гражданским браком - 96,66%

2) вдовцы без детей с низким доходм и целью приобрести недвижимость, действительно существуют. и все 17 человек возвратили кредит в срок. 

---